# 要約 
このJupyter Notebookは、LMSYSのChatbot Arenaコンペティションにおけるチャットボットの応答の好みを予測するための機械学習モデルの開発に取り組んでいます。具体的には、RoBERTaという事前学習済みモデルを用いて、与えられたテキストに対してラベルを予測するタスクを実行しています。

## 問題の概要
Notebookは、チャットボットからの応答を基にしたユーザーの好みを予測するためのモデルを構築することを目的としています。具体的には、トレーニングデータには、各モデルの応答が含まれており、どちらのモデルが好まれるかを予測するためのラベルが与えられています。

## 使用している手法およびライブラリ
1. **ライブラリ**:
   - NumPy: 数値計算
   - pandas: データ操作
   - scikit-learn: モデルの評価（平均二乗誤差など）やデータの分割
   - PyTorch: ニューラルネットワークの構築
   - transformers: 事前学習済みのRoBERTaモデルの使用
   - matplotlib: データの可視化
   - tqdm: プログレスバーの表示

2. **手法**:
   - **データの準備**: 訓練データとテストデータの読み込みと前処理を行い、層化K分割交差検証を使用してデータを分割。
   - **トークナイゼーション**: RoBERTaトークナイザーを用いてテキストデータをトークン化し、モデルに入力できる形式に変換。
   - **モデルの構築**: PyTorchを使用してRoBERTaモデルベースの分類器を構築し、訓練データに基づいて学習。
   - **訓練と評価**: モデルを訓練し（自動混合精度を使用）、検証データで評価するルーチンを実装。
   - **予測**: テストデータに対して予測を実行し、最終的な結果を提出用の形式に整形。

このNotebookは、RoBERTaを利用した深層学習を通じて、ユーザーの好みを予測するモデルを開発するための一連のプロセスを詳しく示しています。最終的に得られた予測結果を元に提出ファイルを生成する準備を行っています。

---


# 用語概説 
以下に、Jupyter Notebookの内容を基に、機械学習・深層学習の初心者がつまずきそうな専門用語についての簡単な解説を列挙します。

1. **torch.cuda.amp.GradScaler**
   - PyTorchの自動混合精度トレーニングで使用するスケーラー。このスケーラーは、計算の精度を自動的に調整し、メモリ消費を減らしつつ、モデルのトレーニングを高速化します。

2. **cuDNN**
   - NVIDIAのGPU向けの深層学習を最適化するためのライブラリ。特に、畳込み層の計算を高速化するために利用され、GPUを使用している場合に速さを向上させる役割を持ちます。

3. **StratifiedKFold**
   - 層化K分割交差検証。データセットをK個のフォールドに分割する際、各フォールドが元のデータセットのクラス分布を維持するように注意して分割します。これにより、クラスの不均衡がある場合でも効果的に評価できます。

4. **tokenizer**  
   - 文章をトークンに分割し、機械が扱いやすい形式に変換するツール。トークニゼーションは自然言語処理において基本的なステップであり、入力文をモデルが理解できる形式にするのに重要です。

5. **attention_mask**
   - トークン化された入力シーケンスにおいて、どのトークンがモデルによって注意を払うべきかを示すマスク。通常、パディングされた部分は0（無視する）とし、実際のトークンには1（処理する）を設定します。

6. **AdamW**
   - Adamオプティマイザの改良版で、重みの減衰（weight decay）を直接オプティマイザに組み込んでいます。これはオーバーフィッティングを防ぐための技術で、特に深層学習において成果を上げています。

7. **get_linear_schedule_with_warmup**
   - 学習率スケジューラで、訓練の最初に学習率を徐々に上げ（ウォームアップ）、その後線形に減少させる方法。これにより、初期の研磨が行われ、学習がより安定します。

8. **RMSE (Root Mean Squared Error)**
   - 平均二乗誤差の平方根を取った指標。予測と実際の値との誤差の大きさを示すもので、モデルの性能評価に使用されます。値が低いほど良いモデルとされます。

9. **DataLoader**
   - PyTorchにおけるデータの入出力管理のためのユーティリティ。バッチ処理やデータのシャッフル、マルチスレッドによるデータの前処理が容易に行えます。

10. **encode_plus**
    - Transformersライブラリで入力テキストをトークン化し、モデルに渡す準備をするためのメソッド。入力トークンID、注意マスク、そしてオプションで追加情報を生成します。

11. **class Dataset**
    - PyTorchにおいてカスタムデータセットを定義するためのクラス。通常、データセットが持つデータやラベルを管理し、データローダーが使いやすい形式で返すメソッド（`__getitem__` や `__len__`）を持っています。

12. **torch.no_grad()**
    - 勾配計算を無効にするためのコンテキストマネージャ。モデルの評価や推論時に使用し、無駄な計算を省くために活用します。

これらの用語は、初心者にとっては親しみのないものもあるため、特に注意が必要です。

---


# LMSYS キーワード torch RoBERTa
インターネット上の条件の下で

https://www.kaggle.com/code/stpeteishii/lmsys-prompt-response-words-keybert

In [ ]:
!pip install chardet  # chardetライブラリをインストールします。 これは、文字エンコーディングを自動的に検出するために使用されます。

In [ ]:
# debug = False  # デバッグモードを無効にします。デバッグ情報は表示されません。
# debug2 = False  # さらなるデバッグモードを無効にします。追加のデバッグ情報は表示されません。

In [ ]:
import numpy as np  # NumPyライブラリをインポートします。数値計算に使います。
import pandas as pd  # pandasライブラリをインポートします。データ操作に使います。
import os  # osモジュールをインポートします。ファイルやディレクトリ操作に使います。
from sklearn.metrics import mean_squared_error  # 平均二乗誤差を計算するための関数をインポートします。
from sklearn.model_selection import StratifiedKFold  # 層化K分割交差検証を行うためのクラスをインポートします。
import torch  # PyTorchライブラリをインポートします。深層学習に使います。
import torch.nn as nn  # PyTorchのニューラルネットワークモジュールをインポートします。
from torch.utils.data import DataLoader, Dataset  # データローダーとデータセットのクラスをインポートします。
from tqdm import tqdm  # 進捗バーを表示するためのtqdmをインポートします。
import matplotlib.pyplot as plt  # グラフ描画のためのmatplotlibをインポートします。
import transformers  # transformersライブラリをインポートします。事前学習済みモデルにアクセスするために使用します。
from transformers import AutoTokenizer, AutoModelForSequenceClassification  # トークナイザーと分類用モデルをインポートします。
import random  # ランダム数生成のためのrandomモジュールをインポートします。
import chardet  # 文字エンコーディングを自動的に検出するためのchardetをインポートします。
import warnings  # 警告メッセージの制御のためにwarningsモジュールをインポートします。
warnings.simplefilter('ignore')  # 警告を無視するフィルターを設定します。
scaler = torch.cuda.amp.GradScaler()  # 自動混合精度トレーニングに使うスケーラーを初期化します。
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 使用可能なデバイスを確認し、CUDA対応GPUがあればそれを使います。
device  # 現在使用しているデバイスを表示します。

In [ ]:
def random_seed(SEED):  # ランダムシードを設定する関数を定義します。
    
    random.seed(SEED)  # randomモジュールのシードを設定します。
    os.environ['PYTHONHASHSEED'] = str(SEED)  # 環境変数にシードを設定します。ハッシュ値の決定に影響します。
    np.random.seed(SEED)  # NumPyの乱数生成器のシードを設定します。
    torch.manual_seed(SEED)  # PyTorch CPUのシードを設定します。
    torch.cuda.manual_seed(SEED)  # PyTorchのCUDA GPUのシードを設定します。
    torch.cuda.manual_seed_all(SEED)  # 全てのCUDAデバイスにシードを設定します。
    torch.backends.cudnn.deterministic = True  # cuDNNの決定論的動作を有効にします。これにより再現性が向上します。
    
SEED = 508  # 使用するシード値を設定します。
random_seed(SEED)  # 定義した関数を呼び出してシードを設定します。

# ラベルはランキングです

In [ ]:
train0 = pd.read_csv('/kaggle/input/lmsys-prompt-response-words-keybert/train_key.csv')  # 訓練データをCSVファイルから読み込みます。
train0 = train0[0:20000]  # 訓練データの最初の20,000行を取得します。
display(train0)  # 訓練データを表示します。
print(train0.columns.tolist())  # 訓練データのカラム名をリスト形式で表示します。

test0 = pd.read_csv('/kaggle/input/lmsys-prompt-response-words-keybert/test_key.csv')  # テストデータをCSVファイルから読み込みます。
display(test0)  # テストデータを表示します。
print(test0.columns.tolist())  # テストデータのカラム名をリスト形式で表示します。

In [ ]:
trainA = train0[['res_a_kw', 'winner_model_a']]  # 訓練データからモデルAに関するカラムを選択します。
trainA.columns = ['text', 'label']  # カラム名を 'text' と 'label' に変更します。
trainB = train0[['res_b_kw', 'winner_model_b']]  # 訓練データからモデルBに関するカラムを選択します。
trainB.columns = ['text', 'label']  # カラム名を 'text' と 'label' に変更します。
data = pd.concat([trainA, trainB], axis=0)  # モデルAとモデルBのデータを縦に結合します。

testA = test0[['res_a_kw']]  # テストデータからモデルAに関するカラムを選択します。
testA['label'] = 0  # モデルAのラベルを0に設定します。
testA.columns = ['text', 'label']  # カラム名を 'text' と 'label' に変更します。
testB = test0[['res_b_kw']]  # テストデータからモデルBに関するカラムを選択します。
testB['label'] = 0  # モデルBのラベルを0に設定します。
testB.columns = ['text', 'label']  # カラム名を 'text' と 'label' に変更します。
TEST = pd.concat([testA, testB], axis=0)  # モデルAとモデルBのテストデータを縦に結合します。

In [ ]:
from sklearn.model_selection import train_test_split  # データを訓練セットとテストセットに分割するための関数をインポートします。
train, test = train_test_split(data, test_size=0.2, random_state=42)  # データを80%の訓練セットと20%のテストセットに分割します。random_stateを設定することで再現性を持たせます。

In [ ]:
# tokenizer = transformers.BertTokenizer.from_pretrained("../input/bert-base-uncased")  # BERTのトークナイザーを事前学習済みモデルから読み込みます。パスは指定された場所に応じて設定します。
tokenizer = transformers.AutoTokenizer.from_pretrained("roberta-base")  # RoBERTaのトークナイザーを事前学習済みモデルから読み込みます。

In [ ]:
test_s = train['text'].iloc[0]  # 訓練データから最初のテキストを取得します。
result1 = tokenizer.encode_plus(test_s)  # テキストをトークン化し、入力IDと注意マスクなどを生成します。
tokenizer.decode(result1["input_ids"])  # トークン化された入力IDをデコードして元のテキストに戻します。

In [ ]:
len(test_s.split(" "))  # テキストをスペースで分割し、単語の数をカウントします。

In [ ]:
result2 = tokenizer.encode_plus(  # テキストをトークン化してエンコードします。
    test_s,  # エンコードするテキスト
    add_special_tokens=True,  # 特殊トークン（[CLS]や[SEP]）を追加します。
    max_length=20,  # 最大トークン数を20に設定します。
    pad_to_max_length=True,  # 最大トークン数に満たない場合はパディングを追加します。
    truncation=True  # 最大長を超える場合はトークンを切り捨てます。
)  # エンコードされた結果をresult2に格納します。

In [ ]:
tokenizer.decode(result2["input_ids"])  # エンコードされた入力IDをデコードして、人間が読める形式のテキストに戻します。

In [ ]:
max_sens = 20  # 最大文の長さを20に設定します。

train = train.sort_values("label").reset_index(drop=True)  # ラベルに基づいて訓練データをソートし、インデックスをリセットします。

train["kfold"] = train.index % 5  # K-Foldクロスバリデーションのために0から4のインデックスを生成します。

p_train = train[train["kfold"] != 0].reset_index(drop=True)  # K-Foldのインデックスが0でない訓練データを取得します。
p_valid = train[train["kfold"] == 0].reset_index(drop=True)  # K-Foldのインデックスが0の訓練データを取得し、検証データとします。

p_test = TEST.reset_index(drop=True)  # テストデータのインデックスをリセットします。

'>>token_type_ids' はRoBERTa/DeBERTaでは必要ありません。

In [ ]:
class BERTDataSet(Dataset):  # BERT用のデータセットクラスを定義します。
    
    def __init__(self, sentences, targets):  # コンストラクタで文とターゲットを受け取ります。        
        self.sentences = sentences  # 文をインスタンス変数に格納します。
        self.targets = targets  # ターゲットをインスタンス変数に格納します。
        
    def __len__(self):  # データセットの長さを返すメソッドを定義します。        
        return len(self.sentences)  # 文の数を返します。
    
    def __getitem__(self, idx):  # インデックス指定でデータを取得するメソッドを定義します。        
        sentence = self.sentences[idx]  # 指定されたインデックスの文を取得します。    
        bert_sens = tokenizer.encode_plus(  # 文をトークン化し、BERT用の形式に変換します。
                                sentence,
                                add_special_tokens=True,  # 特殊トークンを追加します。
                                max_length=max_sens,  # 最大長を設定します。
                                pad_to_max_length=True,  # 最大長にパディングします。
                                return_attention_mask=True)  # 注意マスクも返します。

        ids = torch.tensor(bert_sens['input_ids'], dtype=torch.long)  # 入力IDをテンソルに変換します。
        mask = torch.tensor(bert_sens['attention_mask'], dtype=torch.long)  # 注意マスクをテンソルに変換します。

        target = torch.tensor(self.targets[idx], dtype=torch.float)  # ターゲットをテンソルに変換します。
        
        return {  # 辞書形式でデータを返します。
                'ids': ids,
                'mask': mask,
                'targets': target
            }

In [ ]:
train_dataset = BERTDataSet(p_train["text"], p_train["label"])  # 訓練データセットをBERTDataSetクラスを使って作成します。
valid_dataset = BERTDataSet(p_valid["text"], p_valid["label"])  # 検証データセットを作成します。
test_dataset = BERTDataSet(p_test["text"], p_test["label"])  # テストデータセットを作成します。

train_batch = 16  # 訓練時のバッチサイズを16に設定します。
valid_batch = 32  # 検証時のバッチサイズを32に設定します。
test_batch = 32   # テスト時のバッチサイズを32に設定します。

train_dataloader = DataLoader(train_dataset, batch_size=train_batch, shuffle=True, num_workers=8, pin_memory=True)  # 訓練データローダーを作成します。
valid_dataloader = DataLoader(valid_dataset, batch_size=valid_batch, shuffle=False, num_workers=8, pin_memory=True)  # 検証データローダーを作成します。
test_dataloader = DataLoader(test_dataset, batch_size=test_batch, shuffle=False, num_workers=8, pin_memory=True)  # テストデータローダーを作成します。

In [ ]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=1)  # RoBERTaの事前学習済みモデルを読み込み、シーケンス分類用に設定します。ラベル数は1です。
# model = transformers.BertForSequenceClassification.from_pretrained("../input/bert-base-uncased", num_labels=1)  # BERTの事前学習済みモデルを読み込むためのコメントアウトされたコードです。こちらもラベル数は1です。

In [ ]:
model.to(device)  # モデルを指定したデバイス（GPUまたはCPU）に移動させます。
model.train()  # モデルを訓練モードに設定します。これにより、ドロップアウトなどの訓練専用の機能が有効になります。

In [ ]:
for a in train_dataloader:  # 訓練データローダーからデータを取得するループを開始します。
    ids = a["ids"].to(device)  # バッチ内の入力IDをデバイスに移動させます。
    mask = a["mask"].to(device)  # バッチ内の注意マスクをデバイスに移動させます。
    output = model(ids, mask)  # モデルにIDとマスクを入力し、出力を取得します。
    break  # 一度のループで処理を止めます。 (デバッグ目的についての一時停止)

In [ ]:
output = output["logits"].squeeze(-1).shape  # モデルの出力からlogitsを抽出し、次元を1つ削減して、出力の形状を取得します。

In [ ]:
from transformers import AdamW  # AdamWオプティマイザをインポートします。
LR = 2e-5  # 学習率を2e-5に設定します。
optimizer = AdamW(model.parameters(), LR, betas=(0.9, 0.999), weight_decay=1e-2)  # モデルのパラメータに対してAdamWオプティマイザを初期化します。ベータ値と重み減衰率も設定します。

# エポック数を設定します

In [ ]:
from transformers import get_linear_schedule_with_warmup  # ウォームアップ付きの線形スケジューラをインポートします。
epochs = 30  # エポック数を30に設定します。
# if debug:
#     epochs = 1  # デバッグモードの場合、エポック数を1に設定します（コメントアウトされています）。
train_steps = int(len(p_train) / train_batch * epochs)  # 訓練ステップ数を計算します。
print(train_steps)  # 計算された訓練ステップ数を表示します。
num_steps = int(train_steps * 0.1)  # ウォームアップ期間中のステップ数を全体の10%として計算します。
scheduler = get_linear_schedule_with_warmup(optimizer, num_steps, train_steps)  # オプティマイザ用にウォームアップ付きの線形スケジューラを初期化します。

In [ ]:
def loss_fn(output, target):  # 出力とターゲットに基づいて損失を計算する関数を定義します。
    return torch.sqrt(nn.MSELoss()(output, target))  # 平均二乗誤差（MSE）を計算し、それに平方根を取ってロスを返します。これはRMSE（Root Mean Squared Error）を求めるためです。

# トレーニング（訓練）関数の定義

In [ ]:
def training(  # 訓練を行う関数を定義します。
    train_dataloader,  # 訓練データローダー
    model,  # モデル
    optimizer,  # オプティマイザ
    scheduler  # スケジューラ
):
    
    model.train()  # モデルを訓練モードに設定します。
    torch.backends.cudnn.benchmark = True  # cuDNNのベンチマークを有効にします。動的な最適化のために使用されます。
    allpreds = []  # すべての予測を格納するリスト
    alltargets = []  # すべてのターゲットを格納するリスト

    for a in train_dataloader:  # 訓練データローダーからデータを取得します。

        losses = []  # 各バッチの損失を格納するリスト
        optimizer.zero_grad()  # 勾配を初期化します。

        with torch.cuda.amp.autocast():  # 自動混合精度を使用して計算します。
            
            ids = a["ids"].to(device, non_blocking=True)  # バッチ内の入力IDをデバイスに移動させます。
            mask = a["mask"].to(device, non_blocking=True)  # バッチ内の注意マスクをデバイスに移動させます。

            output = model(ids, mask)  # モデルにIDとマスクを入力し、出力を取得します。
            output = output["logits"].squeeze(-1)  # 出力からlogitsを抽出し、次元を1つ削減します。
            target = a["targets"].to(device, non_blocking=True)  # バッチ内のターゲットをデバイスに移動させます。
            loss = loss_fn(output, target)  # 出力とターゲットを用いて損失を計算します。

            losses.append(loss.item())  # 損失を記録します。
            allpreds.append(output.detach().cpu().numpy())  # 予測をCPUに移動させるとともにリストに追加します。
            alltargets.append(target.detach().squeeze(-1).cpu().numpy())  # ターゲットをCPUに移動させ、リストに追加します。

        scaler.scale(loss).backward()  # 損失をスケールし、逆伝播を行います。
        scaler.step(optimizer)  # オプティマイザでステップ実行します。
        scaler.update()  # スケーラーを更新します。
        
        del loss  # 使用した損失のメモリを解放します。

        scheduler.step()  # スケジューラを1ステップ進めます。

    allpreds = np.concatenate(allpreds)  # すべての予測を結合します。
    alltargets = np.concatenate(alltargets)  # すべてのターゲットを結合します。
    losses = np.mean(losses)  # すべての損失の平均を計算します。
    train_rme_loss = np.sqrt(mean_squared_error(alltargets, allpreds))  # RMSEを計算します。

    return losses, train_rme_loss  # 損失とRMSEを返します。

# 検証（バリデーション）関数の定義

In [ ]:
def validating(valid_dataloader, model):  # 検証を行う関数を定義します。
    
    model.eval()  # モデルを評価モードに設定します。
    allpreds = []  # すべての予測を格納するリスト
    alltargets = []  # すべてのターゲットを格納するリスト

    for a in valid_dataloader:  # 検証データローダーからデータを取得します。
        losses = []  # 各バッチの損失を格納するリスト
        with torch.no_grad():  # 勾配計算を行わないようにします。

            ids = a["ids"].to(device)  # バッチ内の入力IDをデバイスに移動させます。
            mask = a["mask"].to(device)  # バッチ内の注意マスクをデバイスに移動させます。

            output = model(ids, mask)  # モデルにIDとマスクを入力し、出力を取得します。
            output = output["logits"].squeeze(-1)  # 出力からlogitsを抽出し、次元を1つ削減します。
            target = a["targets"].to(device)  # バッチ内のターゲットをデバイスに移動させます。
            loss = loss_fn(output, target)  # 出力とターゲットを用いて損失を計算します。
            losses.append(loss.item())  # 損失を記録します。
            allpreds.append(output.detach().cpu().numpy())  # 予測をCPUに移動させるとともにリストに追加します。
            alltargets.append(target.detach().squeeze(-1).cpu().numpy())  # ターゲットをCPUに移動させ、リストに追加します。
            
            del loss  # 使用した損失のメモリを解放します。

    allpreds = np.concatenate(allpreds)  # すべての予測を結合します。
    alltargets = np.concatenate(alltargets)  # すべてのターゲットを結合します。
    losses = np.mean(losses)  # すべての損失の平均を計算します。
    valid_rme_loss = np.sqrt(mean_squared_error(alltargets, allpreds))  # RMSEを計算します。

    return allpreds, losses, valid_rme_loss  # 予測、損失、RMSEを返します。

    if debug2 == False:  # debug2がFalseの場合の処理
        for a in range(epochs):  # エポック数分のループを開始します。
            for b in train_dataloader:  # 訓練データローダーからデータを取得します。
                break  # 一度のループで処理を止めます。 (デバッグ目的についての一時停止)

        losses, train_rme_loss = training(train_dataloader, model, optimizer, scheduler)  # 訓練を行い、損失とRMSEを取得します。

        for a in valid_dataloader:  # 検証データローダーからデータを取得します。
            break  # 一度のループで処理を止めます。 (デバッグ目的についての一時停止)

# 訓練と検証を行います

In [ ]:
trainlosses = []  # 訓練時の損失を格納するリスト
vallosses = []  # 検証時の損失を格納するリスト
bestscore = None  # 最良スコアを初期化します。
trainscores = []  # 訓練スコアを格納するリスト
validscores = []  # 検証スコアを格納するリスト

for epoch in tqdm(range(epochs)):  # エポック数分のループを進行状況とともに実行します。
    
    print("---------------" + str(epoch) + " start -------------")  # 現在のエポックを表示します。
    
    trainloss, trainscore = training(train_dataloader, model, optimizer, scheduler)  # 訓練を行い、損失とスコアを取得します。
    trainlosses.append(trainloss)  # 訓練損失をリストに追加します。
    trainscores.append(trainscore)  # 訓練スコアをリストに追加します。
    
    print("trainscore is " + str(trainscore))  # 訓練スコアを表示します。
    
    preds, validloss, valscore = validating(valid_dataloader, model)  # 検証を行い、予測、損失とスコアを取得します。
    vallosses.append(validloss)  # 検証損失をリストに追加します。
    validscores.append(valscore)  # 検証スコアをリストに追加します。
    
    print("valscore is " + str(valscore))  # 検証スコアを表示します。
    
    if bestscore is None:  # 最良スコアが未設定の場合
        bestscore = valscore  # 最良スコアを更新します。
        
        print("Save first model")  # 最初のモデルを保存します。
        
        state = {  # モデルの状態を辞書に格納します。
                        'state_dict': model.state_dict(),  # モデルの状態辞書
                        'optimizer_dict': optimizer.state_dict(),  # オプティマイザの状態辞書
                        "bestscore": bestscore  # 最良スコア
                    }
            
        torch.save(state, "model0.pth")  # モデルをファイルに保存します。
        
    elif bestscore > valscore:  # 新しいスコアが最良スコアよりも良い場合
        bestscore = valscore  # 最良スコアを更新します。        
        print("found better point")  # より良いポイントが見つかったことを表示します。        
        state = {  # モデルの状態を辞書に格納します。
                        'state_dict': model.state_dict(),  # モデルの状態辞書
                        'optimizer_dict': optimizer.state_dict(),  # オプティマイザの状態辞書
                        "bestscore": bestscore  # 最良スコア
                    }
            
        torch.save(state, "model0.pth")  # モデルをファイルに保存します。
        
    else:  # それ以外の場合
        pass  # 何もしません。

In [ ]:
plt.scatter(p_valid['label'], preds, alpha=0.2)  # 検証データの実際のラベルと予測値を散布図で表示します。
plt.title('Validation Prediction Result')  # グラフのタイトルを設定します。
plt.xlabel('Actual')  # x軸のラベルを設定します。
plt.ylabel('Prediction')  # y軸のラベルを設定します。
plt.show()  # グラフを表示します。

x = np.arange(epochs)  # エポック数の範囲を生成します。
plt.title('Validation Losses')  # グラフのタイトルを設定します。
plt.xlabel('Epoch')  # x軸のラベルを設定します。
plt.ylabel('Loss')  # y軸のラベルを設定します。
plt.plot(x, trainlosses)  # 訓練損失をプロットします。
plt.plot(x, vallosses)  # 検証損失をプロットします。
plt.show()  # グラフを表示します。

x = np.arange(epochs)  # エポック数の範囲を生成します。
plt.title('Validation Scores')  # グラフのタイトルを設定します。
plt.xlabel('Epoch')  # x軸のラベルを設定します。
plt.ylabel('Score')  # y軸のラベルを設定します。
plt.plot(x, trainscores)  # 訓練スコアをプロットします。
plt.plot(x, validscores)  # 検証スコアをプロットします。
plt.show()  # グラフを表示します。

# モデルの保存

In [ ]:
bestscores = []  # 最良スコアを格納するリストを初期化します。
bestscores.append(bestscore)  # 最初の最良スコアをリストに追加します。

for fold in range(1, 5):  # 1から4までのフォールドについてループします。
    
    # データを初期化します。
    p_train = train[train["kfold"] != fold].reset_index(drop=True)  # 現在のフォールド以外のデータを訓練データに設定します。
    p_valid = train[train["kfold"] == fold].reset_index(drop=True)  # 現在のフォールドのデータを検証データに設定します。

    train_dataset = BERTDataSet(p_train["text"], p_train["label"])  # 訓練データセットの作成。
    valid_dataset = BERTDataSet(p_valid["text"], p_valid["label"])  # 検証データセットの作成。
    
    model = transformers.AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=1)  # RoBERTaの事前学習済みモデルを読み込みます。
    
    model.to(device)  # モデルを指定したデバイスに移動させます。
    LR = 2e-5  # 学習率を設定します。
    optimizer = AdamW(model.parameters(), LR, betas=(0.9, 0.999), weight_decay=1e-2)  # AdamWオプティマイザを初期化します。
    train_steps = int(len(p_train) / train_batch * epochs)  # 訓練ステップ数を計算します。
    num_steps = int(train_steps * 0.1)  # ウォームアップ期間中のステップ数を全体の10%として計算します。
    scheduler = get_linear_schedule_with_warmup(optimizer, num_steps, train_steps)  # スケジューラを初期化します。

    trainlosses = []  # 訓練損失を格納するリスト
    vallosses = []  # 検証損失を格納するリスト
    bestscore = None  # 最良スコアを初期化します。
    trainscores = []  # 訓練スコアを格納するリスト
    validscores = []  # 検証スコアを格納するリスト

    for epoch in tqdm(range(epochs)):  # エポック数分のループを進行状況とともに実行します。

        print("---------------" + str(epoch) + " start -------------")  # 現在のエポックを表示します。

        trainloss, trainscore = training(train_dataloader, model, optimizer, scheduler)  # 訓練を行い、損失とスコアを取得します。
        trainlosses.append(trainloss)  # 訓練損失をリストに追加します。
        trainscores.append(trainscore)  # 訓練スコアをリストに追加します。

        print("trainscore is " + str(trainscore))  # 訓練スコアを表示します。

        preds, validloss, valscore = validating(valid_dataloader, model)  # 検証を行い、予測、損失とスコアを取得します。
        vallosses.append(validloss)  # 検証損失をリストに追加します。
        validscores.append(valscore)  # 検証スコアをリストに追加します。

        print("valscore is " + str(valscore))  # 検証スコアを表示します。

        if bestscore is None:  # 最良スコアが未設定の場合
            bestscore = valscore  # 最良スコアを更新します。

            print("Save first model")  # 最初のモデルを保存します。

            state = {  # モデルの状態を辞書に格納します。
                            'state_dict': model.state_dict(),  # モデルの状態辞書
                            'optimizer_dict': optimizer.state_dict(),  # オプティマイザの状態辞書
                            "bestscore": bestscore  # 最良スコア
                        }

            torch.save(state, "model" + str(fold) + ".pth")  # モデルをファイルに保存します。 

        elif bestscore > valscore:  # 新しいスコアが最良スコアよりも良い場合
            bestscore = valscore  # 最良スコアを更新します。
            print("found better point")  # より良いポイントが見つかったことを表示します。

            state = {  # モデルの状態を辞書に格納します。
                            'state_dict': model.state_dict(),  # モデルの状態辞書
                            'optimizer_dict': optimizer.state_dict(),  # オプティマイザの状態辞書
                            "bestscore": bestscore  # 最良スコア
                        }
            torch.save(state, "model" + str(fold) + ".pth")  # モデルをファイルに保存します。

        else:  # それ以外の場合
            pass  # 何もしません。

    bestscores.append(bestscore)  # 最良スコアをリストに追加します。

In [ ]:
bestscores  # 各フォールドの最良スコアを表示します。

In [ ]:
np.mean(bestscores)  # 各フォールドの最良スコアの平均を計算します。
print("My CV is " + str(np.mean(bestscores)) + ".")  # クロスバリデーションの結果を表示します。

# 予測関数の定義
保存されたモデルは使用しません。

In [ ]:
def predicting(test_dataloader, model):  # 予測を行う関数を定義します。
    
    model.to(device)  # モデルを指定したデバイスに移動させます。
    model.eval()  # モデルを評価モードに設定します。   
    allpreds = []  # すべての予測を格納するリスト
    preds = []  # 個々の予測を格納するリスト
    allvalloss = 0  # 検証損失の合計を初期化します。

    with torch.no_grad():  # 勾配計算を行わないようにします。
        for a in test_dataloader:  # テストデータローダーからデータを取得します。

            ids = a["ids"].to(device)  # バッチ内の入力IDをデバイスに移動させます。
            mask = a["mask"].to(device)  # バッチ内の注意マスクをデバイスに移動させます。

            output = model(ids, mask)  # モデルにIDとマスクを入力し、出力を取得します。
            output = output["logits"].squeeze(-1)  # 出力からlogitsを抽出し、次元を1つ削減します。
            preds.append(output.cpu().numpy())  # 予測をCPUに移動させ、リストに追加します。

        preds = np.concatenate(preds)  # すべての予測を結合します。
        allpreds.append(preds)  # 予測を全体のリストに追加します。

    return allpreds  # すべての予測を返します。

# 予測を行います

In [ ]:
tpreds = predicting(test_dataloader, model)  # テストデータローダーを使用して予測を実行し、結果をtpredsに格納します。

# 予測結果

In [ ]:
test_pred = []  # 予測結果を格納するリストを初期化します。
for p in tpreds[0]:  # tpredsの最初の要素（予測結果）をループします。
    test_pred += [p]  # 各予測値をリストに追加します。

In [ ]:
submit = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')  # 提出用のサンプルCSVファイルを読み込みます。
pa = test_pred[0:len(test0)]  # 予測結果の最初の部分をモデルAの結果として設定します。
pb = test_pred[len(test0):]  # 予測結果の残りの部分をモデルBの結果として設定します。
pc = []  # モデルの引き分け結果を格納するリストを初期化します。
for i in range(len(test0)):  # テストデータの長さ分ループします。
    pc += [np.clip(1 - (pa[i] + pb[i]), 0, 1)]  # モデルAとモデルBの結果から引き分けのスコアを計算し、0〜1の範囲にクリップします。
submit['winner_model_a'] = pa  # 提出データフレームにモデルAの結果を追加します。
submit['winner_model_b'] = pb  # 提出データフレームにモデルBの結果を追加します。
submit['winner_tie'] = pc  # 提出データフレームに引き分けの結果を追加します。
display(submit)  # 提出データフレームを表示します。
submit.to_csv('submission.csv', index=False)  # 提出データフレームをCSVファイルとして保存します。

[Caution] 'submission.csv'は提出用ではありません。このノートブックはインターネット接続の条件で実行されているためです。